In [1]:
import pandas as pd
# from thefuzz import fuzz, process
from rapidfuzz import process, fuzz
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
from concurrent.futures import ProcessPoolExecutor

# AIM: COMPARE and INTEGRATE THE PRODUCTIONS OF 2 DATASET
## Get the overview of the products / market

In [2]:

# Reading the first CSV file and printing its head
df1 = pd.read_csv('./marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv')

# Reading the second CSV file and printing its head
df2 = pd.read_csv('./marketing_sample_for_walmart_com-product_details__20200101_20200331__30k_data.csv')


In [5]:

print(df1['Brand Name'].value_counts())
print(df2['Brand'].value_counts())


Series([], Name: count, dtype: int64)
Brand
Unique Bargains          338
Generic                  310
Anself                   230
Nike                     130
Under Armour             106
                        ... 
Sevylor                    1
Amber Sports               1
J is for Jeep              1
Johnson &amp; Johnson      1
BRITTANIES THYME           1
Name: count, Length: 11382, dtype: int64


In [3]:
# Display options for dataframe
pd.set_option('display.max_rows', 10)  # maximum number of rows to display
pd.set_option('display.max_columns', 10)  # maximum number of columns to display
pd.set_option('display.width', 200)  # width of the display in characters
pd.set_option('display.colheader_justify', 'center')  # center the column headers

print(df1.columns)
print(df2.columns)


Index(['Uniq Id', 'Product Name', 'Brand Name', 'Asin', 'Category', 'Upc Ean Code', 'List Price', 'Selling Price', 'Quantity', 'Model Number', 'About Product', 'Product Specification',
       'Technical Details', 'Shipping Weight', 'Product Dimensions', 'Image', 'Variants', 'Sku', 'Product Url', 'Stock', 'Product Details', 'Dimensions', 'Color', 'Ingredients', 'Direction To Use',
       'Is Amazon Seller', 'Size Quantity Variant', 'Product Description'],
      dtype='object')
Index(['Uniq Id', 'Crawl Timestamp', 'Product Url', 'Product Name', 'Description', 'List Price', 'Sale Price', 'Brand', 'Item Number', 'Gtin', 'Package Size', 'Category', 'Postal Code', 'Available'], dtype='object')


In [7]:
print(df2[['Product Name', 'Category']].head())
print(df2[['Brand', 'List Price', 'Sale Price', 'Gtin', 'Description']].head())

                     Product Name                                         Category                     
0  Allegiance Economy Dual-scale Digital Thermometer  Health | Medicine Cabinet | Thermometers | Dig...
1  Kenneth Cole Reaction Eau De Parfum Spray For ...  Premium Beauty | Premium Fragrance | Premium P...
2  Kid Tough Fitness Inflatable Free-Standing Pun...  Sports & Outdoors | Outdoor Sports | Hunting |...
3                                    THE FIRST YEARS                      Baby | Diapering | Baby Wipes
4  4 Pack - MD USA Seamless Toe-Wave-In Mesh Diab...            Health | Diabetes Care | Diabetic Socks
       Brand        List Price  Sale Price      Gtin                        Description                    
0  Cardinal Health     11.11       11.11    707389636164   We aim to show you accurate product informati...
1     Kenneth Cole     23.99       23.99    191565696101   We aim to show you accurate product informati...
2         BONK FIT     30.76       30.76    85552300

In [8]:
print(df1[['Brand Name', 'List Price', 'Selling Price', 'Upc Ean Code', 'Product Description']].value_counts())


Series([], Name: count, dtype: int64)


In [4]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 10):  # more options can be specified also
    print(df1.count())
    print()
    print(df2.count())




Uniq Id                  10002
Product Name             10002
Brand Name                   0
Asin                         0
Category                  9172
Upc Ean Code                34
List Price                   0
Selling Price             9895
Quantity                     0
Model Number              8230
About Product             9729
Product Specification     8370
Technical Details         9212
Shipping Weight           8864
Product Dimensions         479
Image                    10002
Variants                  2478
Sku                          0
Product Url              10002
Stock                        0
Product Details              0
Dimensions                   0
Color                        0
Ingredients                  0
Direction To Use             0
Is Amazon Seller         10002
Size Quantity Variant        0
Product Description          0
dtype: int64

Uniq Id            30000
Crawl Timestamp    30000
Product Url        30000
Product Name       30000
Description       

In [3]:
def filter_features(df, threshold):
    threshold = threshold * len(df)

# Filter features with more than 80% non-NaN values
    selected_features = df.count()[df.count() > threshold].index.tolist()

    # Print the feature names
    print("Features with more than 80% values:")
    print("\n".join(selected_features))
    return selected_features

In [4]:
filter_features(df1, 0.8)
filter_features(df2, 0.8)

Features with more than 80% values:
Uniq Id
Product Name
Category
Selling Price
Model Number
About Product
Product Specification
Technical Details
Shipping Weight
Image
Product Url
Is Amazon Seller
Features with more than 80% values:
Uniq Id
Crawl Timestamp
Product Url
Product Name
Description
List Price
Sale Price
Brand
Gtin
Category
Available


['Uniq Id',
 'Crawl Timestamp',
 'Product Url',
 'Product Name',
 'Description',
 'List Price',
 'Sale Price',
 'Brand',
 'Gtin',
 'Category',
 'Available']

In [5]:
column_mapping = {
    # Mapping for `df1`
    'Uniq Id': 'Unique ID',
    'Product Name': 'Product Name',
    'Category': 'Category',
    'Selling Price': 'Sale Price',
    'Model Number': 'Model Number',
    'About Product': 'Description',
    'Product Specification': 'Specifications',
    'Technical Details': 'Technical Details',
    'Shipping Weight': 'Shipping Weight',
    'Image': 'Image',
    'Product Url': 'Product URL',
    'Is Amazon Seller': 'Is Amazon Seller',
    
    # Mapping for `df2`
    'Crawl Timestamp': 'Crawl Timestamp',
    'Description': 'Description',
    'List Price': 'List Price',
    'Sale Price': 'Sale Price',
    'Brand': 'Brand Name',
    'Gtin': 'UPC/EAN Code',
    'Available': 'Availability'
}


In [6]:
# Rename columns for df1
df1_cleaned = df1.rename(columns=column_mapping)

# Rename columns for df2
df2_cleaned = df2.rename(columns=column_mapping)


In [70]:
import re

def clean_price_column(price_column):
    def clean_price_range(price_str):
        # Regular expression to match price range (e.g., '74.99 - 249.99')
        match = re.match(r"(\d+(\.\d+)?)\s*-\s*(\d+(\.\d+)?)", price_str.strip())
        
        if match:
            # Extracting the lower and upper price bounds
            lower_price = float(match.group(1))
            upper_price = float(match.group(3))
            return lower_price, upper_price
        else:
            return None, None  # Return None if no valid range is found

    def clean_price(value):
        # Check for price range
        if isinstance(value, str) and ' - ' in value:
            lower_price, upper_price = clean_price_range(value)
            return lower_price, upper_price if lower_price is not None and upper_price is not None else None
        else:
            # Remove common currency symbols and commas
            value = str(value).replace('$', '').replace('€', '').replace('₹', '').replace('£', '').replace(',', '')
            try:
                return float(value)  # Convert to float directly
            except ValueError:
                return None  # In case the value cannot be converted to a float
    
    return price_column.apply(clean_price)


In [71]:
df1_cleaned['Sale Price'] = clean_price_column(df1_cleaned['Sale Price'])
df2_cleaned['Sale Price'] = clean_price_column(df2_cleaned['Sale Price'])

In [9]:
filter_features(df1_cleaned, 0.8)

Features with more than 80% values:
Unique ID
Product Name
Category
Sale Price
Model Number
Description
Specifications
Technical Details
Shipping Weight
Image
Product URL
Is Amazon Seller


['Unique ID',
 'Product Name',
 'Category',
 'Sale Price',
 'Model Number',
 'Description',
 'Specifications',
 'Technical Details',
 'Shipping Weight',
 'Image',
 'Product URL',
 'Is Amazon Seller']

In [7]:

common_features = set(df1_cleaned.columns) & set(df2_cleaned.columns) & set(filter_features(df1_cleaned, 0.8)) & set(filter_features(df2_cleaned, 0.8))
print('Common features:', common_features)

Features with more than 80% values:
Unique ID
Product Name
Category
Sale Price
Model Number
Description
Specifications
Technical Details
Shipping Weight
Image
Product URL
Is Amazon Seller
Features with more than 80% values:
Unique ID
Crawl Timestamp
Product URL
Product Name
Description
List Price
Sale Price
Brand Name
UPC/EAN Code
Category
Availability
Common features: {'Product Name', 'Unique ID', 'Product URL', 'Sale Price', 'Category', 'Description'}


In [8]:
usable_features = common_features - {'Unique ID', 'Product URL', 'Sale Price'}

usable_features = sorted(usable_features, reverse=True)


In [9]:
def preprocess_and_concat(df, common_features):
    df[common_features] = df[common_features].fillna('')
    df['concat'] = df[common_features].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [10]:
preprocess_and_concat(df1_cleaned, list(usable_features))
preprocess_and_concat(df2_cleaned, list(usable_features))
df1_cleaned.columns


Index(['Unique ID', 'Product Name', 'Brand Name', 'Asin', 'Category',
       'Upc Ean Code', 'List Price', 'Sale Price', 'Quantity', 'Model Number',
       'Description', 'Specifications', 'Technical Details', 'Shipping Weight',
       'Product Dimensions', 'Image', 'Variants', 'Sku', 'Product URL',
       'Stock', 'Product Details', 'Dimensions', 'Color', 'Ingredients',
       'Direction To Use', 'Is Amazon Seller', 'Size Quantity Variant',
       'Product Description', 'concat'],
      dtype='object')

In [11]:
# Tokenize category to a dict with index
def tokenize_category(df):
    return df['Category'].apply(lambda x: x.lower().split(' | '))

In [12]:
token2 =tokenize_category(df2_cleaned)
dict2 = dict()
for idx, token in enumerate(token2):
    for cate in token:
        if cate not in dict2:
            dict2[cate] = set()  # Initialize the list if the category is not in the dictionary
        dict2[cate].add(idx)

In [13]:
len(dict2)

4678

In [17]:
df1_cleaned['concat']

0        DB Longboards CoreFlex Crossbow 41" Bamboo Fib...
1        Electronic Snap Circuits Mini Kits Classpack, ...
2        3Doodler Create Flexy 3D Printing Filament Ref...
3        Guillow Airplane Design Studio with Travel Cas...
4        Woodstock- Collage 500 pc Puzzle Make sure thi...
                               ...                        
9997     Remedia Publications REM536B Money Activity Bo...
9998     Trends International NFL La Chargers HG - Mobi...
9999     NewPath Learning 10 Piece Science Owls and Owl...
10000    Disney Princess Do It Yourself Braid Set  Toys...
10001    Hasegawa Ladders Lucano Step Ladder, Orange Ma...
Name: concat, Length: 10002, dtype: object

In [18]:
df1_cleaned['Category'].value_counts()

Category
                                                                                                                                           830
Toys & Games | Games & Accessories | Board Games                                                                                           284
Toys & Games | Puzzles | Jigsaw Puzzles                                                                                                    274
Toys & Games | Stuffed Animals & Plush Toys | Stuffed Animals & Teddy Bears                                                                252
Toys & Games | Toy Figures & Playsets | Action Figures                                                                                     235
                                                                                                                                          ... 
Toys & Games | Hobbies | Remote & App Controlled Vehicles & Parts | Remote & App Controlled Vehicle Parts | Servos & Parts | Servo Ar

In [59]:
token1 =tokenize_category(df1_cleaned)
dict1 = dict()
for idx, token in enumerate(token1):
    for cate in token:
        if cate not in dict1:
            dict1[cate] = set()  # Initialize the list if the category is not in the dictionary
        dict1[cate].add(idx)

In [60]:

fuzzy_matching_list = {}
# Perform fuzzy matching with 2 token dict 1, 2
for token1 in dict1.keys():
    for token2 in dict2.keys():
        score = fuzz.token_set_ratio(token1, token2)
        
        fuzzy_matching_list[(token1, token2)] = score
            


In [ ]:
fuzzy_matching_list

In [ ]:
matching_product_list = []
# Matching Category to the best category in df2
for idx, row in df1_cleaned.iterrows():
    # Tokenize category for df2
    token1 = row['Category'].lower().split(' | ')
    if token1 == ['']:
        continue
    print(idx, token1)
    # Compare with dict1 using fuzzy matching
    common_list = None
    for _, cate in enumerate(token1):
        for dict_cate in dict2.keys():
            score = fuzzy_matching_list[(cate, dict_cate)]
            if score > 80:
                # print(f"{cate} -> {dict_cate} with score {score}")
                if common_list is None:
                    common_list = set(dict2[dict_cate])
                else:
                    common_list = common_list | set(dict2[dict_cate])
                # print("common_list:", common_list)
    if common_list is None:
        continue
    df2_strings = df2_cleaned.loc[list(common_list), 'concat'].tolist()
    top_match = process.extract(row['concat'], df2_strings, scorer=fuzz.token_set_ratio, score_cutoff=50)
    if top_match:
        for index, x in enumerate(top_match):
            top_match[index] = (x[0], x[1], list(common_list)[x[2]])
            # print(f"{row['Product Name']} -> {df2_cleaned['Product Name'][match_index]} with score {score}")
        # print(f"{row['Product Name']} -> {df1_cleaned['Product Name'][match_index]} with score {score}")
        matching_product_list.append((idx, top_match))
        print("===============================================")
        print(f"Matched {len(matching_product_list)} products")
        print("===============================================")

    

0 ['sports & outdoors', 'outdoor recreation', 'skates, skateboards & scooters', 'skateboarding', 'standard skateboards & longboards', 'longboards']
Matched 1 products
1 ['toys & games', 'learning & education', 'science kits & toys']
2 ['toys & games', 'arts & crafts', 'craft kits']
Matched 2 products
3 ['toys & games', 'hobbies', 'models & model kits', 'model kits', 'airplane & jet kits']
4 ['toys & games', 'puzzles', 'jigsaw puzzles']
6 ['clothing, shoes & jewelry', 'costumes & accessories', 'kids & baby', 'girls', 'costumes']
7 ['toys & games', 'arts & crafts', 'drawing & painting supplies', 'crayons']
8 ['home & kitchen', 'home décor', 'window treatments', 'window stickers & films', 'window films']
10 ['toys & games', 'baby & toddler toys']
Matched 3 products
11 ['toys & games', 'collectible toys', 'statues, bobbleheads & busts', 'statues']
12 ['baby products', 'nursery', 'décor', 'window treatments', 'valances']
Matched 4 products
13 ['toys & games', 'building toys', 'building sets

In [63]:
matching_product_list

[(1807,
  [('The Elixir Eatra Large Baby Play Mat Crawling Non-Toxic Playmat, Child Kids Non-Slip Floor Mat, Reversible Waterproof Hyeienic Padded for Toddlers, 82 x 55 x 0.47 inch (made in Korea)  We aim to show you accurate product information. Manufacturers, suppliers and others provide what you see here, and we have not verified it. See our disclaimer |The Elixir Eco Green Extra Large Non-Toxic Reversible Baby Infant Plat Mat Foam Floor Gym Rug The Elixir Eco Green Play Mat provides a safe and comfy place for babies and children to play. The cushioned mat protects against hard falls when kids do what they do best. Approx. 20 Lbs Mat is sturdy so they work well on hardwood floors or carpets. Kids of all ages love the vibrant colors, and in no time theyre identifying numbers and images from the pictures on the mats. FEATURES Babies and children can play on the soft floor surface from birth to school safely. It is waterproof and easy to clean: just wipe with a soft damp cloth for food

In [64]:
len(matching_product_list)

8

In [73]:
for pair in matching_product_list:
    # print(pair)
    df1_idx = pair[0]
    df2_topmatch = pair[1]
    for _, score, df2_idx in df2_topmatch:
        print(score)
        print(df1_cleaned["Product Name"][df1_idx])
        print(df2_cleaned["Product Name"][df2_idx])

        print(df1_cleaned["Sale Price"][df1_idx])
        print(df2_cleaned["Sale Price"][df2_idx])


51.8664047151277
Play with Pieces, Reversible Play Mat, Moroccan Rug Plus Polka Dot
The Elixir Eatra Large Baby Play Mat Crawling Non-Toxic Playmat, Child Kids Non-Slip Floor Mat, Reversible Waterproof Hyeienic Padded for Toddlers, 82 x 55 x 0.47 inch (made in Korea)
149.0
159.0
51.78147268408551
Play with Pieces, Reversible Play Mat, Moroccan Rug Plus Polka Dot
Neat Solutions Disney Winnie the Pooh Meal and Play Mat
149.0
12.95
61.53846153846154
Angeles Toddler White Cot Sheet
KABOER Reusable Kids Children Toddler Foldable Travel Toilet Potty Seat Pad Cushion Mat
15.16
14.76
57.89473684210526
Angeles Toddler White Cot Sheet
(#32) Hotel Quality Silver Grommet Top, Faux Silk 1 Panel Gold Solid Thermal Foam Lined Blackout Heavy Thick Window Curtain Drapes Grommets 95" Length
15.16
7.0
57.89473684210526
Angeles Toddler White Cot Sheet
Mr. Scrappy Custom Colored Sink Flange, White
15.16
37.91
57.89473684210526
Angeles Toddler White Cot Sheet
((#86) Hotel Quality Grommet Top, Jacquard 1 Pan

In [105]:
for idx, match_index, score in matching_product_list:
    print(df2_cleaned["Product Name"][match_index])


Kenneth Cole Reaction Eau De Parfum Spray For Women 3.40 Oz
LIFEWTR, Premium Purified Water, pH Balanced with Electrolytes For Taste, 500 ml bottles (Pack of 12) (Packaging May Vary)
MABIS Vida Mia Digital 20-Second Flexible Tip Childs Duck Thermometer
(Price/CS)Mrs. Millers Spicy Chili Bacon Jam 12/9oz, 571496
Foldable Teepee Tent Kids Classic Play Castle Land Kids Tabernacle Indoor Playhouse Camping Playground Special Offer
Ruffies Pro 42 Gallon Wing Tie 20 count
(Price/CS)Mrs. Millers Spicy Chili Bacon Jam 12/9oz, 571496
Ruffies Pro 42 Gallon Wing Tie 20 count
Ruffies Pro 42 Gallon Wing Tie 20 count
Ruffies Pro 42 Gallon Wing Tie 20 count
Ruffies Pro 42 Gallon Wing Tie 20 count
(Price/CS)Mrs. Millers Spicy Chili Bacon Jam 12/9oz, 571496
Ruffies Pro 42 Gallon Wing Tie 20 count
Ruffies Pro 42 Gallon Wing Tie 20 count
Ruffies Pro 42 Gallon Wing Tie 20 count


In [103]:
len(matching_product_list)

15

In [47]:
df1_cleaned['Category']

0        Sports & Outdoors | Outdoor Recreation | Skate...
1        Toys & Games | Learning & Education | Science ...
2                Toys & Games | Arts & Crafts | Craft Kits
3        Toys & Games | Hobbies | Models & Model Kits |...
4                  Toys & Games | Puzzles | Jigsaw Puzzles
                               ...                        
9997     Toys & Games | Learning & Education | Counting...
9998                          Toys & Games | Arts & Crafts
9999     Office Products | Office & School Supplies | E...
10000            Toys & Games | Arts & Crafts | Craft Kits
10001    Home & Kitchen | Furniture | Kids' Furniture |...
Name: Category, Length: 10002, dtype: object

In [44]:
df2_strings = df2_cleaned['concat'].tolist()

# Match rows in df1 to the best row in df2
matches = []
for idx, row in df1_cleaned['concat'].items():
    # Get the best match and score using rapidfuzz's extractOne
    match, score, match_index = process.extractOne(row, df2_strings, scorer=fuzz.token_sort_ratio)
    matches.append((idx, match_index, score))
    print(f"Match: {df1_cleaned["Product Name"][idx]} -> {df2_cleaned['Product Name'][match_index]}")
    print(score)


Match: DB Longboards CoreFlex Crossbow 41" Bamboo Fiberglass Longboard Complete -> Moose Complete Skateboard STAINED BLUE 7.5" Silver/White ASSEMBLED
51.131401349741964
Match: Electronic Snap Circuits Mini Kits Classpack, FM Radio, Motion Detector, Music Box (Set of 5) -> Just Born Little Dreamer Musical Crib Mobile, Grey Elephant, Zebra, Giraffe
47.9328165374677
Match: 3Doodler Create Flexy 3D Printing Filament Refill Bundle (X5 Pack, Over 1000'. of Extruded Plastics! - Innovate -> Duralite 13 x 13 ft. Safari Party House
48.95522388059701
Match: Guillow Airplane Design Studio with Travel Case Building Kit -> Big Brim Safari Hat-Size:X-Large
47.127784290738575
Match: Woodstock- Collage 500 pc Puzzle -> Listerine Original 500 mL (Pack of 6)
44.688644688644686
Match: Terra by Battat – 4 Dinosaur Toys, Medium – Dinosaurs for Kids & Collectors, Scientifically Accurate & Designed by A Paleo-Artist; Age 3+ (4 Pc) -> 360GT Searchbait Lure
46.6755912539045
Match: Rubie's Child's Pokemon Deluxe

KeyboardInterrupt: 

In [ ]:
def fuzzy_match_products(df1, df2):
    matches = []
    for idx, name in df1[column].items():
        # Find the best match in df2
        
        match = process.extractOne(name, df2[column], scorer=fuzz.partial_ratio)
        print(name)
        print(match)
    return pd.DataFrame(matches)

In [30]:
fuzzy_matches = fuzzy_match_products(df1, df2)
fuzzy_matches

DB Longboards CoreFlex Crossbow 41" Bamboo Fiberglass Longboard Complete
('AMBI', 75, 4288)
Electronic Snap Circuits Mini Kits Classpack, FM Radio, Motion Detector, Music Box (Set of 5)
('Fruit', 80, 191)
3Doodler Create Flexy 3D Printing Filament Refill Bundle (X5 Pack, Over 1000'. of Extruded Plastics! - Innovate
('0', 100, 25663)
Guillow Airplane Design Studio with Travel Case Building Kit
('Salad Dressing', 64, 25173)
Woodstock- Collage 500 pc Puzzle
('0', 100, 25663)
Terra by Battat – 4 Dinosaur Toys, Medium – Dinosaurs for Kids & Collectors, Scientifically Accurate & Designed by A Paleo-Artist; Age 3+ (4 Pc)
('Medium', 100, 28309)
Rubie's Child's Pokemon Deluxe Pikachu Costume, X-Small
('Small', 100, 12989)
Hoffmaster 120813 Double-Tipped Triangular Crayon, 88 mm Length, Wrapped (500 Packs of 2)
('0', 100, 25663)
ARTSCAPE Etched Glass 24" x 36" Window Film, 24-by-36-Inch
('Teeth', 60, 9997)
Pokemon TCG: Sun and Moon Crimson Invasion Elite Trainer Box
('Enter Sandbox', 70, 760)
Mo

KeyboardInterrupt: 